In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import numpy as np

def load_dataset(dataset_dir):
    face_images = []
    face_labels = []

    class_labels = sorted(os.listdir(dataset_dir))
    for class_label in class_labels:
        class_dir = os.path.join(dataset_dir, class_label)
        if not os.path.isdir(class_dir):
            continue

        images = sorted(os.listdir(class_dir))
        for image_name in images:
            image_path = os.path.join(class_dir, image_name)
            if not os.path.isfile(image_path):
                continue

            image = cv2.imread(image_path)
            if image is None:
                continue

            face_images.append(image)
            face_labels.append(class_label)

    return face_images, face_labels

# Provide the dataset directory path
dataset_dir = "/content/data/train"

# Load the dataset
face_images, face_labels = load_dataset(dataset_dir)

# Check the loaded data
print("Total face images:", len(face_images))
print("Total face labels:", len(face_labels))
print("Example face label:", face_labels[0])
print("Example face image shape:", face_images[0].shape)


Total face images: 112
Total face labels: 112
Example face label: Abhinav
Example face image shape: (720, 1280, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
train_dir = "/content/data/train"
test_dir = "/content/data/test"

In [ ]:
image_size = (64, 64)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode="categorical")
test_generator = test_datagen.flow_from_directory(test_dir, target_size=image_size, batch_size=batch_size, class_mode="categorical")

Found 112 images belonging to 4 classes.
Found 49 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'Abhinav': 0, 'Akhil': 1, 'Akshith': 2, 'Dharan': 3}

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(train_generator.num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
num_epochs = 10
model.fit(train_generator, epochs=num_epochs, validation_data=test_generator)

Epoch 1/10
4/4 [==============================] - 3s 415ms/step - loss: 1.3421 - accuracy: 0.4018 - val_loss: 1.2744 - val_accuracy: 0.3061
Epoch 2/10
4/4 [==============================] - 1s 340ms/step - loss: 1.1992 - accuracy: 0.4732 - val_loss: 1.1276 - val_accuracy: 0.3061
Epoch 3/10
4/4 [==============================] - 2s 491ms/step - loss: 1.0656 - accuracy: 0.5179 - val_loss: 0.8982 - val_accuracy: 0.6122
Epoch 4/10
4/4 [==============================] - 2s 568ms/step - loss: 0.7640 - accuracy: 0.7143 - val_loss: 0.5785 - val_accuracy: 0.6735
Epoch 5/10
4/4 [==============================] - 1s 351ms/step - loss: 0.5394 - accuracy: 0.6696 - val_loss: 0.3300 - val_accuracy: 0.9592
Epoch 6/10
4/4 [==============================] - 1s 323ms/step - loss: 0.3323 - accuracy: 0.8750 - val_loss: 0.3225 - val_accuracy: 0.7143
Epoch 7/10
4/4 [==============================] - 1s 314ms/step - loss: 0.2461 - accuracy: 0.8839 - val_loss: 0.1555 - val_accuracy: 1.0000
Epoch 8/10
4/4 [====

In [ ]:
model.save("face_recognition_model.h5")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image


# Testing 1
img1 = image.load_img('/content/data/test/Akhil/Akhil14.jpg',target_size=(64,64))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Abhinav', 'Akhil', 'Akshith', 'Dharan']
print(output[pred])

1/1 [==============================] - 0s 208ms/step
1
Akhil


In [ ]:
# Testing 2
img1 = image.load_img('/content/data/test/Abhinav/Abhinav9.jpg',target_size=(64,64))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Abhinav', 'Akhil', 'Akshith', 'Dharan']
print(output[pred])

1/1 [==============================] - 0s 23ms/step
0
Abhinav


In [ ]:
# Testing 3
img1 = image.load_img('/content/data/test/Akshith/Akshith39.jpg',target_size=(64,64))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Abhinav', 'Akhil', 'Akshith', 'Dharan']
print(output[pred])

1/1 [==============================] - 0s 30ms/step
2
Akshith


In [ ]:
# Testing 4
img1 = image.load_img('/content/data/test/Dharan/Dharan22.jpg',target_size=(64,64))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Abhinav', 'Akhil', 'Akshith', 'Dharan']
print(output[pred])

1/1 [==============================] - 0s 24ms/step
3
Dharan


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("face_recognition_model.h5")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [ ]:
pip install cmake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install dlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install face_recognition==1.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import dlib
import face_recognition
import numpy as np

model = tf.keras.models.load_model('face_recognition_model.h5')

image = face_recognition.load_image_file('/content/data/test/Akshith/Akshith16.jpg')
face_locations = face_recognition.face_locations(image, model="hog")
face_encodings = face_recognition.face_encodings(image, face_locations)

for face_encoding in face_encodings:
    predictions = model.predict(np.expand_dims(face_encoding, axis=0))
    class_index = np.argmax(predictions)
    class_name = train_generator.class_indices[class_index]
    print('Detected face belongs to:', class_name)


In [ ]:
encoded_faces = []
labels = []


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("face_recognition_model.h5")

# Set the path to the directory containing the dataset
data_dir = "/content/data/test"  # Assuming you want to encode the faces from the test directory
label_dir = "/content/data/test"
# Define the image size
image_size = (64, 64)

# Initialize empty lists for storing the encoded faces and labels
encoded_faces = []
labels = []

# Iterate through the directories in the data directory
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        # Iterate through the image files in each label directory
        for filename in os.listdir(label_dir):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                # Load and preprocess the image
                img_path = os.path.join(label_dir, filename)
                img = image.load_img(img_path, target_size=image_size)
                img = image.img_to_array(img)
                img = np.expand_dims(img, axis=0)
                img = img / 255.0  # Normalize the pixel values

                # Encode the face using the trained model
                encoding = model.predict(img)[0]

                # Append the encoding and label to the lists
                encoded_faces.append(encoding)
                labels.append(label)

# Convert the lists to NumPy arrays
encoded_faces = np.array(encoded_faces)
labels = np.array(labels)

# Print the shape of the encoded faces and labels arrays
print("Encoded Faces Shape:", encoded_faces.shape)
print("Labels Shape:", labels.shape)


1/1 [==============================] - 0s 27ms/step
Encoded Faces Shape: (49, 4)
Labels Shape: (49,)
